In [2]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [4]:
%run -i ../../zkstats/core.py

In [5]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [6]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())['col_name']
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))

create_dummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())['col_name']
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

dummy_theory_output = torch.mean(dummy_data_tensor)
theory_output = torch.mean(data_tensor)

In [7]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.tensor(1), torch.mean(X))
        # return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<=torch.abs(0.01*X.size()[1]*self.w), self.w)
    
verifier_define_calculation(dummy_data_path, ['col_name'],sel_dummy_data_path,verifier_model, verifier_model_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_2410/3357874280.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1), torch.mean(X))


In [8]:
# prover calculates settings, send to verifier
# In linearity, scale doesnt affect lookup size
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.tensor(1),torch.mean(X))
        # return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<=torch.abs(0.01*X.size()[1]*self.w), self.w)
prover_gen_settings(data_path,['col_name'], sel_data_path, prover_model,prover_model_path, [15], "resources", settings_path)

Theory_output:  tensor(42.1340)
==== Generate & Calibrate Setting ====
scale:  [15]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":15,"param_scale":15,"scale_rebase_multiplier":10,"lookup_range":[0,100],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":3936,"total_assignments":104,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,30],"model_input_scales":[15],"module_sizes":{"kzg":[],"poseidon":[3936,[1]],"elgamal":[0,[0]]},"required_lookups":[{"Recip":{"scale":32768.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_2410/3070331555.py:10: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1),torch.mean(X))


In [10]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 0.5562019348144531 seconds
Theory output:  tensor(42.1340)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 42.11085656657815
==== Generating Proof ====
proof:  {'instances': [[[1277732943169287410, 11224309302358461726, 16043747437431270579, 166757665308549999], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [3768192631512449736, 6795967255881867556, 15128129630253277759, 52147006479543043]]], 'proof': '086b8507f6f1b90be5514f0c55b49fb3f104089ef562893da8d3b69d2314c5932c4d5fe4c89ed8e2da28ccc15a9371a50a5b0d1195d86b4ebffcc3f08cd4538c0611d3e610167f475676827e00f4b9878d39215e5545dd8eb69720ea2961950f25333e6909f11998e036a2a4d47282716c6507d2b482c880cca0b5e44987bcd72b08383bdc4b15d16e3f91ae9d3511f0668776ff020ffed9ea87788f082726bd0916aa684f16baf0998c674aaeba3a8f0801caa43344e9c23ef6223ae4688ba4200343a4d1f7e4876a800abed13700536446f0d485fdf28104b8994ed6f023a90ac077d

In [12]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[1277732943169287410, 11224309302358461726, 16043747437431270579, 166757665308549999], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [3768192631512449736, 6795967255881867556, 15128129630253277759, 52147006479543043]]]
proof boolean:  1.0
proof result 1 : 42.11085656657815
verified
